![](images/header.png)

---
Data yang tersimpan didalam database MySQL mmepunyai bentuk yang serupa dengan <i>data.frame</i>. MFM kali ini akan membahas penggunaan RDBMS MySQL dengan R. Untuk databasenya sendiri terdiri sembilan tabel yang saling terelasi, dihosting secara online di http://freemysqlhosting.net dan tiap tabel sudah memiliki key masing-masing baik Primary Key atau Foreign Key. Dibawah sudah dilampirkan skema rancangan database <b>sql12258414</b>.

Library atau package yang akan digunakan adalah RMySQL yang memungkinkan kita menuliskan SQL query dan mengeksekusinya dengan R. Adapun RMySQL tergantung/mempunyai dependensi dengan library DBI.

---

### Database "sql12259976"

![database_relationship](images/qqq-1.png)

---

### <i>Load</i> library RMySQL

In [3]:
# install DBI package for R
install.packages("DBI")
# ----------------------------
# install RMySQL package for R
install.packages("RMySQL")
library(RMySQL)

Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.5:
  cannot open URL 'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.5/PACKAGES'"Warning message:
"package 'DBI' is in use and will not be installed"Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.5:
  cannot open URL 'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.5/PACKAGES'"Warning message:
"package 'RMySQL' is in use and will not be installed"

### Buat koneksi dengan database sql12259976

In [4]:
mydb = dbConnect(MySQL(), 
    user='sql12259976', 
    password='9ap7vT4XSb', 
    dbname='sql12259976', 
    host='sql12.freemysqlhosting.net')

### Tampilkan tabel yang terdapat didalam database "sql12259976"

In [5]:
dbListTables(mydb)

[1] "actor"           "director"        "genres"          "movie"          
[5] "movie_cast"      "movie_direction" "movie_genres"    "rating"         
[9] "reviewer"

### Tampilkan field yang terdapat didalam tabel "movie"

In [14]:
dbListFields(mydb, 'movie')

[1] "act_id"     "act_fname"  "act_lname"  "act_gender"

---
## SELECT
---

### Select basic - Tampilkan 6 data teratas dari tabel "movie"

In [7]:
str.query = dbSendQuery(mydb, "SELECT * FROM movie")
movie.data = fetch(str.query, n=-1)
head(movie.data)

mov_id,mov_title,mov_year,mov_time,mov_lang,mov_dt_rel,mov_rel_country
901,Vertigo,1958,128,English,1958-08-24,UK
902,The Innocents,1961,100,English,1962-02-19,SW
903,Lawrence of Arabia,1962,216,English,1962-12-11,UK
904,The Deer Hunter,1978,183,English,1979-03-08,UK
905,Amadeus,1984,160,English,1985-01-07,UK
906,Blade Runner,1982,117,English,1982-09-09,UK


![select 1](images/select1.png)

### Select basic - Tampilkan 6 judul film teratas beserta negara rilisnya

In [8]:
str.query = dbSendQuery(mydb, "SELECT mov_title, mov_rel_country FROM movie")
movie.data = fetch(str.query, n=-1)
head(movie.data)

mov_title,mov_rel_country
Vertigo,UK
The Innocents,SW
Lawrence of Arabia,UK
The Deer Hunter,UK
Amadeus,UK
Blade Runner,UK


![select 2](images/select2.png)

### Select basic - Tampilkan 6 data teratas dari tabel "actor"

In [9]:
str.query = dbSendQuery(mydb, "SELECT act_fname, act_lname FROM actor")
movie.data = fetch(str.query, n=-1)
head(movie.data)

act_fname,act_lname
James,Stewart
Deborah,Kerr
Peter,OToole
Robert,De Niro
F. Murray,Abraham
Harrison,Ford


![select 3](images/select3.png)

### Select basic - Tampilkan nama lengkap para aktor yang mempunyai nama belakang yang berawalan huruf "A"

In [13]:
str.query = dbSendQuery(mydb, "SELECT CONCAT(act_fname,' ',act_lname) AS full_name FROM actor WHERE act_lname LIKE 'A%' ")
movie.data = fetch(str.query, n=-1)
head(movie.data)
#count it

full_name
F. Murray Abraham
Woody Allen
David Aston
Ali Astin


![select 4](images/select4.png)

### Select basic - Tampilkan nama lengkap aktor perempuan yang mempunyai nama belakang yang berawalan huruf "A"

In [15]:
str.query = dbSendQuery(mydb, "SELECT CONCAT(act_fname,' ',act_lname) FROM actor WHERE act_lname LIKE 'A%' AND act_gender = 'f' ")
movie.data = fetch(str.query, n=-1)
head(movie.data)

"CONCAT(act_fname,' ',act_lname)"
Ali Astin


![select 5](images/select5.png)

### Select basic - Hitung banyaknya film yang dirilis sebelum tahun 1990

In [16]:
str.query = dbSendQuery(mydb, "SELECT COUNT(*) FROM movie WHERE mov_year < 1990")
movie.data = fetch(str.query, n=-1)
head(movie.data)

COUNT(*)
12


![select 6](images/select6.png)

### Select basic - Tampilkan film yang dirilis sebelum film "Blade Runner"

In [21]:
# NOTES
# 1. Gunakan  sub-query
# 2. Lakukan tanpa langsung menuliskan tahun rilis Blade Runner
str.query = dbSendQuery(mydb, "SELECT * FROM `movie` WHERE mov_year < (SELECT mov_year from movie WHERE mov_title = 'Blade Runner')")
movie.data = fetch(str.query, n=-1)
head(movie.data)

mov_id,mov_title,mov_year,mov_time,mov_lang,mov_dt_rel,mov_rel_country
901,Vertigo,1958,128,English,1958-08-24,UK
902,The Innocents,1961,100,English,1962-02-19,SW
903,Lawrence of Arabia,1962,216,English,1962-12-11,UK
904,The Deer Hunter,1978,183,English,1979-03-08,UK
909,Chinatown,1974,130,English,1974-08-09,UK
911,Annie Hall,1977,93,English,1977-04-20,USA


![select 7](images/select7.png)

## JOIN
---

![joins](images/joins.png)

### Step up - Inner join

In [22]:
str.query = dbSendQuery(mydb, "SELECT 
                                    * 
                                FROM 
                                    `director` a 
                                JOIN movie_direction b ON a.dir_id = b.dir_id ")
movie.data = fetch(str.query, n=-1)
movie.data

dir_id,dir_fname,dir_lname,dir_id,mov_id
304,Mel,Gibson,304,925
306,Jack,Clayton,306,903
308,Michael,Cimino,308,904
309,Hayao,Miyazaki,309,912
309,Hayao,Miyazaki,309,927
310,Milos,Forman,310,905
311,Akira,Kurosawa,311,926
312,Ridley,Scott,312,906
313,James,Cameron,313,915
313,James,Cameron,313,922


### Step up - Left join

In [23]:
str.query = dbSendQuery(mydb, "SELECT 
                                    * 
                                FROM 
                                    `director` a 
                                LEFT JOIN movie_direction b ON a.dir_id = b.dir_id ")
movie.data = fetch(str.query, n=-1)
movie.data

dir_id,dir_fname,dir_lname,dir_id,mov_id
303,Alfred,Hitchcock,NA,NA
304,Mel,Gibson,304,925
305,David,Lean,NA,NA
306,Jack,Clayton,306,903
307,Robert,Zemeckis,NA,NA
308,Michael,Cimino,308,904
309,Hayao,Miyazaki,309,912
309,Hayao,Miyazaki,309,927
310,Milos,Forman,310,905
311,Akira,Kurosawa,311,926


### Step up - Right join

In [24]:
str.query = dbSendQuery(mydb, "SELECT 
                                    * 
                                FROM 
                                    movie_direction b
                                RIGHT JOIN `director` a  ON b.dir_id = a.dir_id ")
movie.data = fetch(str.query, n=-1)
movie.data

dir_id,mov_id,dir_id,dir_fname,dir_lname
NA,NA,303,Alfred,Hitchcock
304,925,304,Mel,Gibson
NA,NA,305,David,Lean
306,903,306,Jack,Clayton
NA,NA,307,Robert,Zemeckis
308,904,308,Michael,Cimino
309,912,309,Hayao,Miyazaki
309,927,309,Hayao,Miyazaki
310,905,310,Milos,Forman
311,926,311,Akira,Kurosawa


### Step up - Tampilkan nama sutradara beserta dengan film yang disutradarainya

In [26]:
str.query = dbSendQuery(mydb, "SELECT 
                                    CONCAT(a.dir_fname,' ',a.dir_lname) AS 'Full Name', c.mov_title AS 'Movie Title' 
                                FROM 
                                    `director` a 
                                JOIN movie_direction b ON a.dir_id = b.dir_id 
                                JOIN movie c ON b.mov_id = c.mov_id")
movie.data = fetch(str.query, n=-1)
movie.data

Full Name,Movie Title
Mel Gibson,Braveheart
Jack Clayton,Lawrence of Arabia
Michael Cimino,The Deer Hunter
Hayao Miyazaki,Princess Mononoke
Hayao Miyazaki,Spirited Away
Milos Forman,Amadeus
Akira Kurosawa,Seven Samurai
Ridley Scott,Blade Runner
James Cameron,Titanic
James Cameron,Aliens


![stepup 1](images/stepup1.png)

### Step up - Tampilkan semua judul film beserta ratingnya

In [33]:
str.query = dbSendQuery(mydb, "SELECT 
                                    a.mov_title, b.rev_stars,
                                    CASE 
                                        WHEN b.rev_stars < 5 THEN 'Under rated'
                                        WHEN b.rev_stars >= 5 AND b.rev_stars <= 8 THEN 'Good Movie'
                                        WHEN b.rev_stars > 8 THEN 'A Blockbuster!' 
                                    END AS 'Comment'
                                FROM 
                                    movie a
                                LEFT JOIN rating b ON a.mov_id = b.mov_id")
movie.data = fetch(str.query, n=-1)
head(movie.data)

mov_title,rev_stars,Comment
Vertigo,8,Good Movie
The Innocents,8,Good Movie
Lawrence of Arabia,8,Good Movie
The Deer Hunter,NA,NA
Amadeus,NA,NA
Blade Runner,8,Good Movie


![stepup 2](images/stepup2.png)

### Step up - Susun rating dari yang tertinggi ke terendah dari hasil query sebelumnya

In [35]:
str.query = dbSendQuery(mydb, "SELECT 
                                    mov_title, rev_stars,
                                    CASE 
                                        WHEN b.rev_stars < 5 THEN 'Under rated'
                                        WHEN b.rev_stars BETWEEN 5 AND 8 THEN 'Good movie'
                                        WHEN b.rev_stars > 8 THEN 'A Blockbuster!' 
                                    END AS 'Comment'
                                FROM 
                                    movie a 
                                LEFT JOIN rating b ON a.mov_id = b.mov_id
                                ORDER BY rev_stars DESC")
movie.data = fetch(str.query, n=-1)
movie.data

mov_title,rev_stars,Comment
The Usual Suspects,9,A Blockbuster!
Lawrence of Arabia,8,Good movie
Blade Runner,8,Good movie
Aliens,8,Good movie
Braveheart,8,Good movie
Princess Mononoke,8,Good movie
Titanic,8,Good movie
The Innocents,8,Good movie
Slumdog Millionaire,8,Good movie
Annie Hall,8,Good movie


![stepup3](images/stepup3.png)

### Step up - Hitung jumlah komentar

In [36]:
str.query = dbSendQuery(mydb, "SELECT 
                                    CASE 
                                        WHEN b.rev_stars < 5 THEN 'Under rated'
                                        WHEN b.rev_stars BETWEEN 5 AND 8 THEN 'Good movie'
                                        WHEN b.rev_stars > 8 THEN 'A Blockbuster!' 
                                    END AS 'Comment',
                                    COUNT(*) AS Counter
                                FROM 
                                    movie a
                                LEFT JOIN rating b ON a.mov_id = b.mov_id
                                GROUP BY Comment
                                ORDER BY Counter DESC")
movie.data = fetch(str.query, n=-1)
movie.data

Comment,Counter
Good movie,14
NA,9
Under rated,4
A Blockbuster!,1


![stepup 4](images/stepup4.png)

### Step up - Join dengan menggunakan <i>WHERE</i>

In [37]:
str.query = dbSendQuery(mydb, "SELECT 
                                    CASE 
                                        WHEN b.rev_stars < 5 THEN 'Under rated'
                                        WHEN b.rev_stars >= 5 AND b.rev_stars <= 8 THEN 'Good Movie'
                                        WHEN b.rev_stars > 8 THEN 'A Blockbuster!' 
                                    END AS 'Comment',
                                    COUNT(*) AS Counter
                                FROM 
                                    movie a, rating b
                                WHERE
                                    a.mov_id = b.mov_id
                                GROUP BY Comment
                                ORDER BY Counter DESC")
movie.data = fetch(str.query, n=-1)
movie.data

Comment,Counter
Good Movie,14
Under rated,4
A Blockbuster!,1


### Step up - UNION

In [38]:
str.query = dbSendQuery(mydb, "SELECT 
                                    CONCAT(act_fname,' ',act_lname) AS 'Full Name',
                                    'Actor' AS Role
                                FROM 
                                    actor
                                UNION
                                SELECT 
                                    CONCAT(dir_fname,' ',dir_lname) AS 'Full Name',
                                    'Director' AS Role
                                FROM 
                                    director")
movie.data = fetch(str.query, n=-1)
movie.data

Full Name,Role
James Stewart,Actor
Deborah Kerr,Actor
Peter OToole,Actor
Robert De Niro,Actor
F. Murray Abraham,Actor
Harrison Ford,Actor
Nicole Kidman,Actor
Stephen Baldwin,Actor
Jack Nicholson,Actor
Mark Wahlberg,Actor
